In [17]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU,Dense
import numpy as np
# from nltk.translate import meteor_score
# from nltk.tokenize import word_tokenize
import tensorflow.keras.backend as K

In [2]:
batch_size = 64
epochs = 10
latent_dim = 256
num_samples = 20000

In [3]:
data_path = 'fra.txt'

In [4]:
input_texts = []
output_texts = []
input_characters = set()
output_characters = set()

with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, output_text, _ = line.split('\t')
    
    # We use 'tab' as the 'start sequence' character
    # for the targets, and '\n' as the 'end sequence' character.
    
    output_text = '\t' + output_text + '\n'
    input_texts.append(input_text)
    output_texts.append(output_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in output_text:
        if char not in output_characters:
            output_characters.add(char)


In [5]:
len(output_characters)

99

In [6]:
input_characters = sorted(list(input_characters))
output_characters = sorted(list(output_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in output_texts])

In [7]:
print("Number of Samples:", len(input_texts))
print('Number of unique input Tokens:' , num_encoder_tokens)
print('Number of unique output Tokens:' , num_decoder_tokens)
print('Max sequence length for inputs:' , max_encoder_seq_length)
print('Max sequence length for outputs:' , max_decoder_seq_length)

Number of Samples: 20000
Number of unique input Tokens: 74
Number of unique output Tokens: 99
Max sequence length for inputs: 17
Max sequence length for outputs: 59


In [8]:
input_token_index = dict(
    [(char,i) for i,char in enumerate(input_characters)])

output_token_index = dict(
    [(char,i) for i,char in enumerate(output_characters)])

In [9]:
input_token_index , output_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '/': 10,
  '0': 11,
  '1': 12,
  '2': 13,
  '3': 14,
  '4': 15,
  '5': 16,
  '6': 17,
  '7': 18,
  '8': 19,
  '9': 20,
  ':': 21,
  '?': 22,
  'A': 23,
  'B': 24,
  'C': 25,
  'D': 26,
  'E': 27,
  'F': 28,
  'G': 29,
  'H': 30,
  'I': 31,
  'J': 32,
  'K': 33,
  'L': 34,
  'M': 35,
  'N': 36,
  'O': 37,
  'P': 38,
  'Q': 39,
  'R': 40,
  'S': 41,
  'T': 42,
  'U': 43,
  'V': 44,
  'W': 45,
  'Y': 46,
  'a': 47,
  'b': 48,
  'c': 49,
  'd': 50,
  'e': 51,
  'f': 52,
  'g': 53,
  'h': 54,
  'i': 55,
  'j': 56,
  'k': 57,
  'l': 58,
  'm': 59,
  'n': 60,
  'o': 61,
  'p': 62,
  'q': 63,
  'r': 64,
  's': 65,
  't': 66,
  'u': 67,
  'v': 68,
  'w': 69,
  'x': 70,
  'y': 71,
  'z': 72,
  'é': 73},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '%': 5,
  '&': 6,
  "'": 7,
  ',': 8,
  '-': 9,
  '.': 10,
  '0': 11,
  '1': 12,
  '2': 13,
  '3': 14,
  '4': 15,
  '5': 16,
  '6': 17,
  '7'

In [10]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length , num_decoder_tokens), dtype = 'float32')
decoder_output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length,num_decoder_tokens) , dtype = 'float32')

In [11]:
for i,(input_text,output_text) in enumerate(zip(input_texts,output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1.
    
    for t, char in enumerate(output_text):
        # decoder_output_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i,t,output_token_index[char]] = 1.
        if t>0:
            # decoder_output_data will be ahead by one timestep
            # and will not include the start character
            decoder_output_data[i,t-1,output_token_index[char]] = 1.
    decoder_input_data[i,t+1:, output_token_index[' ']] = 1.
    decoder_output_data[i , t: , output_token_index[' ']] = 1.

In [12]:
encoder_input_data[0].shape

(17, 74)

In [13]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = GRU(latent_dim, return_state=True)
encoder_outputs, state = encoder(encoder_inputs)
encoder_states = [state]

In [15]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, state = decoder_gru(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [19]:
# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_output_data'
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)

#Run training:
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy' , metrics = ['accuracy', f1_score])
model.fit([encoder_input_data,decoder_input_data], decoder_output_data,
         batch_size=batch_size,
         epochs = epochs,
         validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 33s 122ms/step - loss: 0.4857 - accuracy: 0.8550 - f1_score: 0.8752 - val_loss: 0.5797 - val_accuracy: 0.8263 - val_f1_score: 0.8530
Epoch 2/10
250/250 [==============================] - 29s 118ms/step - loss: 0.4730 - accuracy: 0.8586 - f1_score: 0.8780 - val_loss: 0.5717 - val_accuracy: 0.8299 - val_f1_score: 0.8568
Epoch 3/10
250/250 [==============================] - 45s 179ms/step - loss: 0.4607 - accuracy: 0.8621 - f1_score: 0.8809 - val_loss: 0.5593 - val_accuracy: 0.8345 - val_f1_score: 0.8595
Epoch 4/10
250/250 [==============================] - 51s 204ms/step - loss: 0.4480 - accuracy: 0.8657 - f1_score: 0.8841 - val_loss: 0.5429 - val_accuracy: 0.8401 - val_f1_score: 0.8624
Epoch 5/10
250/250 [==============================] - 53s 213ms/step - loss: 0.4330 - accuracy: 0.8702 - f1_score: 0.8881 - val_loss: 0.5283 - val_accuracy: 0.8442 - val_f1_score: 0.8655
Epoch 6/10
250/250 [==============================] - 58s 231ms/s

In [ ]:
# define sampling models:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_input_states = [decoder_state_input_h,decoder_state_input_c]

decoder_outputs,state_h,state_c = decoder_gru(decoder_inputs, initial_state=decoder_input_states)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs]+decoder_input_states,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i,char) for char,i in input_token_index.items())
reverse_output_char_index = dict(
    (i,char) for char,i in output_token_index.items())

def decode_sequence(input_seq):
    states_value= encoder_model.predict(input_seq)
    output_seq = np.zeros((1,1,num_decoder_tokens))
    output_seq[0,0,output_token_index['\t']] = 1
    
    stop_condition = False
    decoded_sentences = ''
    while not stop_condition:
        output_tokens,h,c = decoder_model.predict(
            [output_seq]+ states_value)
        
        sampled_token_index = np.argmax(output_tokens[0,-1, :])
        sampled_char = reverse_output_char_index[sampled_token_index]
        decoded_sentences += sampled_char

        if(sampled_char == '\n' or len(decoded_sentences) > max_decoder_seq_length):
            stop_condition = True
        
        #update the target sequence (of length 1):
        output_seq = np.zeros((1,1,num_decoder_tokens))
        output_seq[0,0,sampled_token_index] = 1
        
        states_value = [h,c]
    return decoded_sentences

for seq_index in range(20):
    # take one sequence for trying out decoding:
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentences = decode_sequence(input_seq)
    print('ProjectGurukul Project: English to French Translation ')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:' , decoded_sentences)

1/1 [==============================] - 0s 20ms/step
ProjectGurukul Project: English to French Translation 
Input sentence: Go.
Decoded sentence: Prends un conter.

1/1 [==============================] - 0s 23ms/step
ProjectGurukul Project: English to French Translation 
Input sentence: Go.
Decoded sentence: Prends un conter.

1/1 [==============================] - 0s 21ms/step
ProjectGurukul Project: English to French Translation 
Input sentence: Go.
Decoded sentence: Prends un conter.

1/1 [==============================] - 0s 21ms/step
ProjectGurukul Project: English to French Translation 
Input sentence: Go.
Decoded sentence: Prends un conter.

1/1 [==============================] - 0s 20ms/step
ProjectGurukul Project: English to French Translation 
Input sentence: Hi.
Decoded sentence: Quile est chande !

1/1 [==============================] - 0s 20ms/step
ProjectGurukul Project: English to French Translation 
Input sentence: Hi.
Decoded sentence: Quile est chande !

1/1 [=========

In [ ]:
# def preprocess_and_tokenize(sentence):
#     sentence = sentence.replace('\t', '').replace('\n', '')
#     tokens = word_tokenize(sentence.lower())
#     return tokens

In [ ]:
# meteor_references = []
# meteor_hypotheses = []

# for seq_index in range(20):
#     input_seq = encoder_input_data[seq_index:seq_index + 1]
#     decoded_sentence = decode_sequence(input_seq)

#     input_tokens = preprocess_and_tokenize(input_texts[seq_index])
#     decoded_tokens = preprocess_and_tokenize(decoded_sentence)

#     print('Input sentence:', ' '.join(input_tokens))
#     print('Decoded sentence:', ' '.join(decoded_tokens))

#     meteor_references.append(input_tokens)
#     meteor_hypotheses.append(decoded_tokens)

#     print('-' * 20)

# meteor = meteor_score.corpus_meteor_score(meteor_references, meteor_hypotheses)
# print('Overall METEOR Score:', meteor)

1/1 [==============================] - 0s 20ms/step


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Nhat Thanh/nltk_data'
    - 'c:\\Users\\Nhat Thanh\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data'
    - 'c:\\Users\\Nhat Thanh\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data'
    - 'c:\\Users\\Nhat Thanh\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data'
    - 'C:\\Users\\Nhat Thanh\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
